This notebook implements the CNN model Resnet34 without cross-validation. Refer to the notebook 'Pretrained resnet34 Wetlands Cross Validation.ipynb' to view the code for the model with cross validation.

# **Enable GPU support**

Follow these steps to enable free GPU support on Google Colab for training the model faster

1. Click Runtime in menu bar
2. Select Change Runtime type
3. Select GPU and click Okay

https://www.tutorialspoint.com/google_colab/google_colab_using_free_gpu.htm


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Display images in Wetland directory

In [ ]:
!ls "/content/drive/My Drive/Wetland_nonwetland_dataset/Image_Data/Wetland"

 EPSG3857_Date20190619_Lat31.152683_Lon-97.386207_Mpp0.037.jpg
 EPSG3857_Date20190619_Lat31.152757_Lon-97.386474_Mpp0.037.jpg
 EPSG3857_Date20190619_Lat31.15293_Lon-97.386233_Mpp0.037.jpg
 EPSG3857_Date20190619_Lat31.155379_Lon-97.376547_Mpp0.037.jpg
 EPSG3857_Date20190619_Lat31.155619_Lon-97.376199_Mpp0.037.jpg
 EPSG3857_Date20190619_Lat31.155634_Lon-97.376536_Mpp0.037.jpg
 EPSG3857_Date20190619_Lat31.155647_Lon-97.376852_Mpp0.037.jpg
 EPSG3857_Date20190619_Lat31.155942_Lon-97.376593_Mpp0.037.jpg
 EPSG3857_Date20190619_Lat31.156657_Lon-97.380565_Mpp0.037.jpg
 EPSG3857_Date20190619_Lat31.15688_Lon-97.380629_Mpp0.037.jpg
 EPSG3857_Date20190619_Lat31.156904_Lon-97.380987_Mpp0.037.jpg
 EPSG3857_Date20190619_Lat31.156908_Lon-97.380323_Mpp0.037.jpg
 EPSG3857_Date20190619_Lat31.157127_Lon-97.380669_Mpp0.037.jpg
 EPSG3857_Date20190619_Lat31.159789_Lon-97.484253_Mpp0.037.jpg
 EPSG3857_Date20190619_Lat31.160302_Lon-97.488751_Mpp0.037.jpg
 EPSG3857_Date20190619_Lat31.160421_Lon-97.485723_Mpp0.03

Display images in Non-wetland directory

In [ ]:
!ls "/content/drive/My Drive/Wetland_nonwetland_dataset/Image_Data/Nonwetland"

 EPSG3857_Date20190619_Lat31.15245_Lon-97.386215_Mpp0.037.jpg
 EPSG3857_Date20190619_Lat31.152739_Lon-97.385843_Mpp0.037.jpg
 EPSG3857_Date20190619_Lat31.152757_Lon-97.386903_Mpp0.037.jpg
 EPSG3857_Date20190619_Lat31.153011_Lon-97.386507_Mpp0.037.jpg
 EPSG3857_Date20190619_Lat31.153108_Lon-97.385789_Mpp0.037.jpg
 EPSG3857_Date20190619_Lat31.155006_Lon-97.376373_Mpp0.037.jpg
 EPSG3857_Date20190619_Lat31.155177_Lon-97.376948_Mpp0.037.jpg
 EPSG3857_Date20190619_Lat31.155509_Lon-97.375917_Mpp0.037.jpg
 EPSG3857_Date20190619_Lat31.155675_Lon-97.37732_Mpp0.037.jpg
 EPSG3857_Date20190619_Lat31.156104_Lon-97.375904_Mpp0.037.jpg
 EPSG3857_Date20190619_Lat31.156367_Lon-97.381067_Mpp0.037.jpg
 EPSG3857_Date20190619_Lat31.156419_Lon-97.380232_Mpp0.037.jpg
 EPSG3857_Date20190619_Lat31.156861_Lon-97.38122_Mpp0.037.jpg
 EPSG3857_Date20190619_Lat31.156886_Lon-97.379978_Mpp0.037.jpg
 EPSG3857_Date20190619_Lat31.157011_Lon-97.380013_Mpp0.037.jpg
 EPSG3857_Date20190619_Lat31.157292_Lon-97.381157_Mpp0.037

Count Nonwetland Images

In [ ]:
import os
FOLDER_PATH_1 = 'Nonwetland'
ROOT_PATH = '/content/drive/My Drive/Wetland_nonwetland_dataset/Image_Data/'
print(len(os.listdir(os.path.join(ROOT_PATH, FOLDER_PATH_1))))

188


Count Wetland Images

In [ ]:
FOLDER_PATH_2 = 'Wetland'
ROOT_PATH = '/content/drive/My Drive/Wetland_nonwetland_dataset/Image_Data/'
print(len(os.listdir(os.path.join(ROOT_PATH, FOLDER_PATH_2))))

180


Hence, we have a balanced dataset

#**Packages Installed**

**glob**: allows us to retrieve paths of data inside sub folders easily

**open-cv**: is used as the image processing library to read and preprocess images

**numpy**: is used for matrix operations

**torch**: is used to create the Dataset and Dataloader classes, and for converting data to tensors.

**albumentations** - transform and augment images into tensor

When running the cell below, you will need to click into the output and type "y" twice, whenever there is a output like Proceed (y/n) appears

In [ ]:
!pip install numpy
!pip install torch
!pip install glob
!pip install -U albumentations

!pip uninstall opencv-python
!pip uninstall opencv-contrib-python
!pip uninstall opencv-contrib-python-headless
!pip install opencv-contrib-python==4.5.5.62


In [ ]:
import glob
import shutil
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, Subset, TensorDataset, random_split, ConcatDataset
from torchvision import transforms, utils
from torch.utils.data.sampler import SubsetRandomSampler, WeightedRandomSampler
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models

#**Image Augmentation**

https://medium.com/nanonets/how-to-use-deep-learning-when-you-have-limited-data-part-2-data-augmentation-c26971dc8ced 

https://blog.roboflow.com/image-augmentations-for-aerial-datasets/ 

https://blog.roboflow.com/why-and-how-to-implement-random-crop-data-augmentation/


https://towardsdatascience.com/how-to-implement-augmentations-for-multispectral-satellite-images-segmentation-using-fastai-v2-and-ea3965736d1

**Image augmentations for train and test data**

In [ ]:
from albumentations.augmentations.transforms import Flip
train_transforms = A.Compose([
    #A.RandomCrop(width=256, height=256),
    A.Resize(450, 450),
    A.Downscale (scale_min=0.25, scale_max=0.25, interpolation=0, always_apply=False, p = 1),
    A.Blur(blur_limit=3),
    A.GaussianBlur (blur_limit=(3, 7), sigma_limit=0, always_apply=False, p = 1),
    A.Flip(p=1),
    A.ColorJitter (brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, always_apply=False, p=1),
    A.RandomBrightnessContrast(p=1),
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

# https://towardsdatascience.com/custom-dataset-in-pytorch-part-1-images-2df3152895

#**Custom Dataloader**

In [ ]:
class CustomDataset(Dataset):
    '''
    Create a custom image dataset class that also assigns labels to each image based
    on root directory label (Wetland or Nonwetland).
    '''
    def __init__(self, transform=None):
        self.imgs_path = "/content/drive/My Drive/Wetland_nonwetland_dataset/Image_Data/"
        file_list = glob.glob(self.imgs_path + "*")
        self.data = []
        for class_path in file_list:
            class_name = class_path.split("/")[-1]
            for img_path in glob.glob(class_path + "/*.jpg"):
                self.data.append([img_path, class_name])
        self.class_map = {"Nonwetland" : 0, "Wetland": 1}
        # Apply transformation to image
        self.transform = transform 

    def __len__(self):
      '''
      Returns size of dataset
      '''  
        return len(self.data)

    def __getitem__(self, idx):
      '''
      Returns an image from the dataset given an index like dataset[i] can be 
      used to get i-th image.
      '''
        img_path, class_name = self.data[idx]
        image = Image.open(img_path)
        image = np.array(image)
        class_id = self.class_map[class_name]
        class_id = torch.tensor([class_id]) # label to tensor
        if self.transform is not None: 
            image_tensor = self.transform(image=image)["image"]
        return image_tensor, class_id


    def target(self):
      '''
      Returns a list of target labels
      '''
      target_labels = torch.empty(len(self.data))
      for i in range(len(self.data)):
        _, target = self.data[i]
        target_labels[i] = self.class_map[target]
      return target_labels
      

# https://medium.com/analytics-vidhya/creating-a-custom-dataset-and-dataloader-in-pytorch-76f210a1df5d
# https://towardsdatascience.com/custom-dataset-in-pytorch-part-1-images-2df3152895 

**Image and its label as a tensor**

In [ ]:
image_dataset = CustomDataset(train_transforms)
img_tensor, label_tensor = image_dataset[180]
print("image tensor", img_tensor)
print("label tensor", label_tensor)

# img_tensor.size()

image tensor tensor([[[ 1.4813,  1.4813,  1.3614,  ..., -1.1559, -1.1559, -1.1559],
         [ 1.4813,  1.4813,  1.3614,  ..., -1.1559, -1.1559, -1.1559],
         [ 1.5840,  1.5840,  1.4642,  ..., -1.1902, -1.1902, -1.1902],
         ...,
         [-0.3853, -0.3853, -0.4196,  ..., -0.0599, -0.0599, -0.0599],
         [-0.3853, -0.3853, -0.4196,  ..., -0.0599, -0.0599, -0.0599],
         [-0.3853, -0.3853, -0.4196,  ..., -0.0599, -0.0599, -0.0599]],

        [[ 0.3764,  0.3764,  0.3239,  ..., -1.2868, -1.2868, -1.2868],
         [ 0.3764,  0.3764,  0.3239,  ..., -1.2868, -1.2868, -1.2868],
         [ 0.5165,  0.5165,  0.4639,  ..., -1.2868, -1.2868, -1.2868],
         ...,
         [-0.8141, -0.8141, -0.8316,  ..., -0.1488, -0.1313, -0.1313],
         [-0.8141, -0.8141, -0.8316,  ..., -0.1488, -0.1313, -0.1313],
         [-0.8141, -0.8141, -0.8316,  ..., -0.1488, -0.1313, -0.1313]],

        [[ 0.9499,  0.9499,  0.8802,  ..., -1.2636, -1.2636, -1.2636],
         [ 0.9499,  0.9499,  0.8

#**Split images into training and testing data**

In [ ]:
def train_test_split(img_dataset):
  '''
  Uses custom image dataset object to instantiate a Dataloader (PyTorch) object
  while for training and testing dataset.
  Returns: training and testing dataloader 
  '''
  test_split = .2
  random_seed= 42

  # Creating data indices for training and validation splits
  dataset_size = len(images)
  indices = list(range(dataset_size))
  split = int(np.floor(test_split * dataset_size))
  np.random.seed(random_seed)
  np.random.shuffle(indices)
  train_indices, test_indices = indices[split:], indices[:split]


  target = images.target().numpy() # array of target labels for each sample
  target = target.astype(int) # array of integers
  train_labels = [target[x] for x in train_indices]
  test_labels = [target[x] for x in test_indices]

  # Pytorch train data sampler
  class_sample_count_train = np.unique(train_labels, return_counts=True)[1] # array of counts of each label
  weights_train = 1. / class_sample_count_train # array of weights for each unique label
  samples_weight_train = np.array([weights_train[x] for x in train_labels]) # array of weights for each sample
  samples_weight_train = torch.from_numpy(samples_weight_train) # tensor of weights for each sample
  samples_weight_train = samples_weight_train.double()
  train_sampler = WeightedRandomSampler(samples_weight_train, len(samples_weight_train), replacement = True)
  
  # Pytorch test data sampler
  class_sample_count_test = np.unique(test_labels, return_counts=True)[1] # array of counts of each label
  weights_test = 1. / class_sample_count_test # array of weights for each unique label
  samples_weight_test = np.array([weights_test[x] for x in test_labels]) # array of weights for each sample
  samples_weight_test = torch.from_numpy(samples_weight_test) # tensor of weights for each sample
  samples_weight_test = samples_weight_test.double()
  test_sampler = WeightedRandomSampler(samples_weight_test, len(samples_weight_test), replacement = True)

  # Pytorch dataloaders train and test
  trainloader = DataLoader(Subset(images, train_indices), sampler = train_sampler, batch_size = 25, drop_last = True, num_workers=2)
  testloader = DataLoader(Subset(images, test_indices), sampler = test_sampler, batch_size = 25, drop_last = True)

  return trainloader, testloader

trainloader, testloader = train_test_split(image_dataset)
loaders = {'train' : trainloader, 'test'  : testloader}

# https://stackoverflow.com/questions/50544730/how-do-i-split-a-custom-dataset-into-training-and-test-datasets/50544887#50544887
# Implementation of testloader with WeightedRandomSampler: https://discuss.pytorch.org/t/how-to-handle-imbalanced-classes/11264/40
# https://discuss.pytorch.org/t/how-to-handle-imbalanced-classes/11264

Another way to implement train-test split with scikit-learn: https://stackoverflow.com/questions/50544730/how-do-i-split-a-custom-dataset-into-training-and-test-datasets/50544887#50544887


#**Using a pre-trained Model ResNet34 (a.k.a Transfer Learning)**

1. Load in pre-trained weights from a convolutional neural network trained on a large dataset

2. Freeze all the weights in the lower (convolutional) layers: the layers to freeze are adjusted depending on similarity of new task to original dataset

3. Replace the upper layers of the network with a custom classifier: the number of outputs must be set equal to the number of classes. When the extra layers are added to the model, they are set to trainable by default (require_grad=True). Only changing the very last original fully-connected layer.

4. Train only the custom classifier layers for the task thereby optimizing the model for smaller dataset

https://debuggercafe.com/satellite-image-classification-using-pytorch-resnet34/ 


https://towardsdatascience.com/transfer-learning-with-convolutional-neural-networks-in-pytorch-dd09190245ce 


# **Compare different pre-trained models**

I compared Resnet34 and Resnet101 accuracies and training times for 45 epochs and the former had higher accuracy and much lower training time. Hence, I implemented Resnet34 model.

https://learnopencv.com/pytorch-for-beginners-image-classification-using-pre-trained-models/

**Resources**

How to choose layers for convolutional neural network architecture
[link text](https://www.google.com/search?q=how+to+choose+layers+for+convolutional+neural+network+architecture&oq=how+to+choose+layers+for+convolutional&aqs=chrome.4.69i57j33i160l3j33i22i29i30j33i15i22i29i30.13710j0j7&sourceid=chrome&ie=UTF-8 )

The purpose of dropout layers and batch normalization layers in a CNN model are ways of reducing overfitting of the model

In [ ]:
# Instantiate pre-trained model Resnet34
model = models.resnet34(pretrained=True)

# Freeze model weights from the pre-trained model
for param in model.parameters():
    param.requires_grad = False

# Add a custom classifier as the final layer/fully connected layer of pre-trained model
# change the final classification head, it is trainable
num_classes = 2
model.fc = nn.Linear(512, num_classes)

# changes all model params to float64 which is same as double
model = model.double() 

print(model)


/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# **Shift the model to GPU**

In [ ]:
# check if CUDA is available
use_cuda = torch.cuda.is_available()

# move model to GPU if CUDA is available
if use_cuda:
    model = model.cuda()

# **Total parameters and trainable parameters**

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.\n")


21,285,698 total parameters.
1,026 training parameters.



#**Loss function and Optimizer**

In [ ]:
# learning rate
lr = 0.001
# optimizer
optimizer = optim.Adam(model.parameters(), lr = lr)
# loss function
criterion = nn.CrossEntropyLoss()

#**Model Training Function**

In [ ]:
# Shows progress bar
from tqdm.auto import tqdm

def train(model, trainloader, optimizer, criterion):
    '''
    Train the Resnet34 model using trainloader object's images.
    Returns: Training loss and training accuracy.
    '''

    model.train()
    print('Training')
    train_running_loss = 0.0
    train_running_correct = 0
    counter = 0
    for i, data in tqdm(enumerate(trainloader), total=len(trainloader)):
        counter += 1
        image, labels = data
        if use_cuda:
          image = image.cuda()
        if use_cuda:
          labels = labels.cuda()
        image = image.double()
        labels = labels.squeeze(1)
        # empty the gradients
        optimizer.zero_grad()
        # forward pass
        outputs = model(image)
        #print("label", labels.size(), labels)
        # calculate the loss
        loss = criterion(outputs, labels)
        train_running_loss += loss.item()
        # calculate the accuracy
        _, preds = torch.max(outputs.data, 1)
        train_running_correct += (preds == labels).sum().item()
        # backpropagation
        loss.backward()
        # update the optimizer parameters
        optimizer.step()

    # loss and accuracy for the complete epoch
    epoch_loss = train_running_loss / counter
    epoch_acc = 100. * (train_running_correct / len(trainloader.dataset))
    return epoch_loss, epoch_acc, model, optimizer

#**Load a sample of test data**

In [ ]:
eval_samples = []
for samples, labels in loaders['test']:
    eval_samples.append([samples, labels])

#**Model Validation Function**

In [ ]:
def validate(model, eval_samples, criterion, class_names):
    '''
    Test the model using testloader object's images.
    Returns testing loss and testing accuracy from one epoch.
    '''
    
    model.eval()
    print('Validation')
    valid_running_loss = 0.0
    valid_running_correct = 0
    counter = 0
    
    # two lists to track class-wise accuracy
    class_correct = list(0. for i in range(len(class_names)))
    class_total = list(0. for i in range(len(class_names)))

    with torch.no_grad():
        for i, data in tqdm(enumerate(eval_samples), total = len(testloader)):
            counter += 1
            image = data[0]
            labels = data[1]
            if use_cuda:
              image = image.cuda()
            if use_cuda:
              labels = labels.cuda()
            labels = labels.squeeze(1)
            image = image.double()
            # forward pass
            image = image.double()
            outputs = model(image)
            # calculate the loss
            loss = criterion(outputs, labels)
            valid_running_loss += loss.item()
            # calculate the accuracy
            _, preds = torch.max(outputs.data, 1)
            valid_running_correct += (preds == labels).sum().item()

            # calculate the accuracy for each class
            correct  = (preds == labels).squeeze()
            for i in range(len(preds)):
                label = labels[i]
                class_correct[label] += correct[i].item()
                class_total[label] += 1
        
    # loss and accuracy for the complete epoch
    epoch_loss = valid_running_loss / counter
    epoch_acc = 100. * (valid_running_correct / len(testloader.dataset))
    
    # print the accuracy for each class after every epoch
    print('\n')
    for i in range(len(class_names)):
        print(f"Accuracy of class {class_names[i]}: {100*class_correct[i]/class_total[i]}")
    print('\n')

    return epoch_loss, epoch_acc, model

#**Create directories to store checkpoint and best model**

https://towardsdatascience.com/how-to-save-and-load-a-model-in-pytorch-with-a-complete-example-c2920e617dee 

In [ ]:
%cd "/content/drive/My Drive/Wetland_nonwetland_dataset/"
!ls

/content/drive/My Drive/Wetland_nonwetland_dataset
0  4			       checkpoint_resnet101	    Image_Data
1  best_model_resnet101        checkpoint_resnet34	    User_Interface_Test
2  best_model_resnet101_64X4D  checkpoint_resnext101_64X4D
3  best_model_resnet34	       flagged


In [ ]:
%mkdir checkpoint_resnet34 best_model_resnet34

mkdir: cannot create directory ‘checkpoint_resnet34’: File exists
mkdir: cannot create directory ‘best_model_resnet34’: File exists


**Save model checkpoint**

In [ ]:
def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    To save checkpoint, the latest one and the best one. This creates flexibility: 
    either you are interested in the state of the latest checkpoint or the best checkpoint.
    state: checkpoint we want to save
    is_best: is this the best checkpoint
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

**Load the saved model**

In [ ]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    Load a saved Pytorch model.
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    Returns the model.
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    return model

**Specify file paths**

In [ ]:
checkpoint_path = "/content/drive/My Drive/Wetland_nonwetland_dataset/checkpoint_resnet34/current_checkpoint.pt"
best_model_path = "/content/drive/My Drive/Wetland_nonwetland_dataset/best_model_resnet34/best_model.pt"

#**Model training and testing begins across 45 epochs!**

In [ ]:
epochs = 45
valid_loss_min = np.Inf

dataset_classes = ["Wetland", "Nonwetland"]
# track of losses and accuracies for train and test data
train_loss, valid_loss = [], []
train_acc, valid_acc = [], []
# start the training
for epoch in range(epochs):
    print(f"[INFO]: Epoch {epoch+1} of {epochs}")
    train_epoch_loss, train_epoch_acc, model, optimizer = train(model, trainloader, 
                                              optimizer, criterion)
    valid_epoch_loss, valid_epoch_acc, model = validate(model, eval_samples,  
                                                 criterion, dataset_classes)
    train_loss.append(train_epoch_loss)
    valid_loss.append(valid_epoch_loss)
    train_acc.append(train_epoch_acc)
    valid_acc.append(valid_epoch_acc)
    print(f"Training loss: {train_epoch_loss:.3f}, training acc: {train_epoch_acc:.3f}")
    print(f"Validation loss: {valid_epoch_loss:.3f}, validation acc: {valid_epoch_acc:.3f}")
    print('-'*50)

    # create checkpoint variable and add important data
    checkpoint = {
            'epoch': epochs,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }
    # save the model if validation loss has decreased
    if valid_epoch_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min, valid_epoch_loss))
        # save checkpoint as best model
        save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = valid_epoch_loss

print('TRAINING COMPLETE')

[INFO]: Epoch 1 of 45
Training


  0%|          | 0/11 [00:00<?, ?it/s]

Validation


  0%|          | 0/2 [00:00<?, ?it/s]



Accuracy of class Wetland: 38.46153846153846
Accuracy of class Nonwetland: 62.5


Training loss: 0.743, training acc: 44.068
Validation loss: 0.679, validation acc: 34.247
--------------------------------------------------
Validation loss decreased (inf --> 0.679011).  Saving model ...
[INFO]: Epoch 2 of 45
Training


...



Accuracy of class Wetland: 76.92307692307692
Accuracy of class Nonwetland: 70.83333333333333


Training loss: 0.405, training acc: 76.271
Validation loss: 0.478, validation acc: 50.685
--------------------------------------------------
[INFO]: Epoch 45 of 45
Training


  0%|          | 0/11 [00:00<?, ?it/s]

Validation


  0%|          | 0/2 [00:00<?, ?it/s]



Accuracy of class Wetland: 69.23076923076923
Accuracy of class Nonwetland: 83.33333333333333


Training loss: 0.371, training acc: 78.644
Validation loss: 0.500, validation acc: 52.055
--------------------------------------------------
TRAINING COMPLETE


In [ ]:
best_model = load_ckp(best_model_path, model, optimizer)
best_model

#**Gradio Web Interface for Application**


[Google Colab Gradio Pytorch Custom Model](https://www.google.com/search?q=google+colab+gradio+pytorch+custom+model&sxsrf=ALiCzsaEt_avbosFnlvM6HX8GsafqIxPLA%3A1658558444716&ei=7JfbYvGqK62HptQPg5KVuA4&ved=0ahUKEwjx0L7Xs475AhWtg4kEHQNJBecQ4dUDCA4&uact=5&oq=google+colab+gradio+pytorch+custom+model&gs_lcp=Cgdnd3Mtd2l6EAMyBwghEAoQoAEyBQghEKsCOggIIRAeEBYQHUoECEEYAUoECEYYAFDpB1iZGWDMI2gBcAB4AIABd4gB7gqSAQQxLjEymAEAoAEBwAEB&sclient=gws-wiz)



[Gradio app video tutorial](https://www.youtube.com/watch?v=wruyZWre2sM)

[Gradio and Hugging Face](https://www.youtube.com/watch?v=b1NgUiTIUMc)

[Google Colab code for PyTorch](https://colab.research.google.com/drive/1S6seNoJuU7_-hBX5KbXQV4Fb_bbqdPBk?usp=sharing#scrollTo=kd8zTAkbHqJV)

[load a directory of examples in the interface](https://gradio.app/more_on_examples_and_flagging/)

In [ ]:
!pip install gradio

In [ ]:
model = best_model.eval()

def predict_fn(img):
  '''
  Add docstring
  Input: (img) the input image as a PIL image
  Returns: (confidences) the predictions, as a dictionary whose keys are 
    class labels and whose values are confidence probabilities
  '''
  labels = ["Nonwetland", "Wetland"]
  preprocess = transforms.Compose((transforms.ToPILImage(),
                                     transforms.Resize(size = (450, 450)),
                                     transforms.ToTensor()))
  img = torch.unsqueeze(preprocess(img), 0) 

  img = img.double()
  if use_cuda:
      img = img.cuda()
  prediction = torch.nn.functional.softmax(model(img)[0], dim=0)
  confidences = {labels[i]: float(prediction[i]) for i in range(2)}    
  return confidences

# Image() which creates the drag-and-drop input component and converts image to a NumPy array.
# The output component will be a Label, which displays the top 2 prediction 
# probabilities for the image by constructing it as Label(num_top_classes=2).
# The examples parameter allows to prepopulate our interfaces with a few predefined examples.

import gradio as gr

gr.Interface(fn=predict_fn, 
             inputs=gr.Image(),
             outputs=gr.Label(num_top_classes=2),
             title='Identify Wetlands in Boston').launch(debug=True)

# https://gradio.app/image_classification_in_pytorch/
# https://blog.scaleway.com/how-to-make-a-demo-of-your-machine-learning-model-with-gradio/ 

/content/drive/My Drive/Wetland_nonwetland_dataset/User_Interface_Test
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://53514.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


https://gradio.app/image_classification_in_pytorch/

https://discuss.pytorch.org/t/runtimeerror-mat1-and-mat2-shapes-cannot-be-multiplied-64x13056-and-153600x2048/101315/13

https://discuss.pytorch.org/t/how-to-lower-the-dimension-of-the-tensor-in-pytorch/41478

https://towardsdatascience.com/a-guide-to-an-efficient-way-to-build-neural-network-architectures-part-ii-hyper-parameter-42efca01e5d7

https://betterprogramming.pub/computer-vision-how-to-set-up-your-cnn-architecture-229c014db7fb 

https://stackoverflow.com/questions/24509921/how-do-you-decide-the-parameters-of-a-convolutional-neural-network-for-image-cla 